In [2]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir('../notebook_format')
from formats import load_style
load_style()

In [79]:
os.chdir(path)
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Naive Bayes

**Naive Bayes** classifiers is based on Bayes’ theorem, and the adjective naive comes from the assumption that the features in a dataset are mutually independent. In practice, the independence assumption is often violated, but **Naive Bayes** still tend to perform very well in the fields of text/document classification. Common applications includes spam filtering (categorized a text message as spam or not-spam) and sentiment analysis (categorized a text message as positive or negative review). More importantly, the simplicity of the method means that it takes order of magnitude less time to train when compared to more complext models like support vector machines.


## Text/Document Representations

Text classifiers often don’t use any kind of deep representation about language: often times a document is represented as a bag of words. (A bag is like a set that allows repeating elements.) This is an extremely simple representation as it throws away the word order and only keeps which words are included in the document and how many times each word occurs.
 
We shall look at two probabilistic models of documents, both of which represent documents as a bag of words, using the **Naive Bayes** assumption. Both models represent documents using feature vectors
whose components correspond to word types. If we have a document containing $|V|$ distinct vocabularies,
then the feature vector dimension $d=|V|$.

- **Bernoulli document model:** a document is represented by a feature vector with binary elements taking value 1 if the corresponding word is present in the document and 0 if the word is not present.
- **Multinomial document model:** a document is represented by a feature vector with integer elements whose value is the frequency of that word in the document.

Example: Consider the vocabulary V = {blue,red, dog, cat, biscuit, apple}. In this case |V| = d = 6. Now consider the (short) document "the blue dog ate a blue biscuit". If $d^B$ is the **Bernoulli** feature vector for this document, and $d^M$ is the **Multinomial** feature vector, then we would have:

In [4]:
vocab = [ 'blue', 'red', 'dog', 'cat', 'biscuit', 'apple' ]
doc = "the blue dog ate a blue biscuit"

# note that the words that didn't appear in the vocabulary will be discarded
bernoulli = [ 1 if v in doc else 0 for v in vocab ]
multinomial = [ doc.count(v) for v in vocab ]
print( 'bernoulli', bernoulli )
print( 'multinomial', multinomial )

bernoulli [1, 0, 1, 0, 1, 0]
multinomial [2, 0, 1, 0, 1, 0]


## Bernoulli Model

Consider a document $D$, whose class is given by $C = 1, 2, ..., K$. We classify D as the class which has the highest posterior probability $argmax_{ k = 1, 2, ..., K} \, P(C = k|D)$, which can be re-expressed using Bayes’ Theorem:

$$p(C = k|D) = \frac{ p(C = k) \, p(D|C = k) }{p(D)} \ \propto p(C = k) \, p(D|C = k)$$

Where:

- $\propto$ means is proportional to.
- $p(C = k)$ represents the class k's prior probabilities.
- $p(D|C = k)$ is the likelihoods of the document given the class k.
- $p(D)$ is the noramlizing factor which we don't have to compute since it does not depend on the class $C$, or meaning that the it's same for all class $C$.

Starting with $p(D|C)$. The spirit of **Naive Bayes** is it assumes that each of the features it uses are conditionally independent of one another given some class. More formally, if we wish to calculate the probability of observing features $X_1$ through $X_d$, given some class $C$ we can do it by the following math formula:

$$p(x_{1},x_{2},...,x_{d} \mid C) = \prod_{i=1}^{d}p(x_{i} \mid C)$$ 

Now if we have a vocabulary (features) $V$ containing a set of $|V|$ words and the $t_{th}$ dimension of a document vector corresponds to word $w_t$ in the vocabulary. If we follow the **Naive Bayes** assumption, that the probability of each word occurring in the document is independent of the occurrences of the other words, then we can re-write the $i_{th}$ document likelihood $p(D_i \mid C)$ as:

$$p(D_i \mid C ) = \prod_{t=1}^{d}b_{it}p(w_t \mid C) + ( 1 - b_{it} ) (1- p(w_t \mid C)) $$

Where:

- $p(w_t \mid C)$ is the probability of word $w_t$ occurring in a document of class $C$.
- $1- p(w_t \mid C)$ is the probability of $w_t$ not occurring in a document of class $C$.
- $b_{it}$ is either 0 or 1 representing the absence or presence of word $w_t$ in the $i_{th}$ document.

This product goes over all words in the vocabulary. If word $w_t$ is present, then $b_{it} = 1$ and the associated probability is $p(w_t \mid C)$; If word $w_t$ is not present, then $b_{it} = 0$ and the associated probability becomes $1- p(w_t \mid C)$.


As for the word likelihood $p(w_t \mid C)$, we can learn (estimate) these parameters from a training set of documents labelled with class $C=k$.

$$p(w_t \mid C = k) = \frac{n_k(w_t)}{N_k}$$

Where:

- $n_k(w_t)$ is the number of class $C=k$'s document in which $w_t$ is observed.
- $N_k$ is the number of documents that belongs to class $k$.

Last, calculating $p(C)$ is relatively simple: If there are $N$ documents in total in the training set, then the prior probability of class $C=k$ may be estimated as the relative frequency of documents of class $C=k$:

$$p(C = k)\,= \frac{N_k}{N}$$

Where $N$ is the total number of documents in the training set.

## Bernoulli Model Example

Consider a set of documents, each of which is related either to Class 1 or to Class 0. Given
a training set of 11 documents, we would like to train a Naive Bayes classifier, using the Bernoulli
document model, to classify unlabelled documents as Class 1 or 0. We define a vocabulary of eight words.

Thus the training data $X$ is presented below as a 11*8 matrix, in which each row represents an 8-dimensional document vector. And the $y$ represents the class of each document. Then we would like to classify the two testing data.

In [63]:
train = np.genfromtxt( 'train.txt', dtype = np.int )
X_train = train[ :, :-1 ]
y_train = train[ :, -1 ] # the last column is the class
print('training data:')
print(X_train)
print()
print(y_train)
print()
print('testing data:')
X_test = np.array([ [1, 0, 0, 1, 1, 1, 0, 1], [0, 1, 1, 0, 1, 0, 1, 0] ])
print(X_test)

training data:
[[1 0 0 0 1 1 1 1]
 [0 0 1 0 1 1 0 0]
 [0 1 0 1 0 1 1 0]
 [1 0 0 1 0 1 0 1]
 [1 0 0 0 1 0 1 1]
 [0 0 1 1 0 0 1 1]
 [0 1 1 0 0 0 1 0]
 [1 1 0 1 0 0 1 1]
 [0 1 1 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 1 0]]

[1 1 1 1 1 1 0 0 0 0 0]

testing data:
[[1 0 0 1 1 1 0 1]
 [0 1 1 0 1 0 1 0]]


In [77]:
def bernoulli_nb( X_train, y_train, X_test ):
    """
    Pass in the training data, it's label and 
    predict the testing data's class
    """
    
    # calculate the prior proabability p(C=k)
    N = X_train.shape[0]
    priors = np.bincount(y_train) / N
    
    # obtain the unique class's type (since it may not be 0 and 1)
    class_type = np.unique(y_train)
    class_nums = class_type.shape[0]
    word_likelihood = np.zeros( ( class_nums, X_train.shape[1] ) )
    
    # calculate the word likelihood p(w_t∣C)
    for index, output in enumerate(class_type):
        subset = X_train[ np.equal( y_train, output ) ]
        word_likelihood[ index, : ] = np.sum( subset, axis = 0 ) / subset.shape[0]
        
    # make predictions on the test set
    # note that this code will break if there's only one test set
    # since the first for loop will not be looping through each document
    # but each document's feature
    predictions = np.zeros( X_test.shape[0], dtype = np.int )
    for index1, document in enumerate(X_test):

        posteriors = np.zeros(class_nums)
        
        # calculate p(C = k|D) for the document for all class
        # and return the predicted class with the maximum probability
        for c in range(class_nums):
            
            # start with p(C = k)
            posterior = priors[c]
            word_likelihood_subset = word_likelihood[ c, : ]
            
             # loop through features to calculate p(D∣C = k)
            for index2, feature in enumerate(document):

                if feature:
                    prob = word_likelihood_subset[index2]
                else:
                    prob = 1 - word_likelihood_subset[index2]
                posterior *= prob

            posteriors[c] = posterior

        predicted_class = class_type[ np.argmax(posteriors) ]
        predictions[index1] = predicted_class
    
    return predictions

In [78]:
predictions = bernoulli_nb( X_train, y_train, X_test )
predictions

array([1, 0])

## Multinomial Distribution

Before discussing the multinomial document model, it is important to be familiar with the multinomial
distribution. We first need to be able to count the number of distinct arrangements of a set of items, when some of the items are indistinguishable. For example: Using all the letters, how many distinct sequences can you make from the word "Mississippi"? There are 11 letters to permute, but "i" and "s" occur four times, "p" twice and "M" once. This gives the total number distinct permutations as:

$$\frac{11!}{4!4!2!1!}$$

Generally if we have $n$ items of $d$ types, such that $n_1 + n_2 + ... + n_d = n$, then the number of distinct permutations is given by:

$$\frac{n!}{n_1!n_2!...n_d!}$$

Suppose a population contains items of $d \geq 2$ different types and that the proportion of items that
are of type $t$ is $p_t, t=1, ..., d$, with

$$\sum_{t=1}^d p_t =1 \\\, p_t > 0, \text{for all } t$$

Then the probability of 

If all of that is still unclear, try this [Youtube Video on Introduction to the Multinomial Distribution](https://www.youtube.com/watch?v=syVW7DgvUaY).

## Multinomial Model

In the multinomial document model, the document feature vectors capture the frequency of words, not
just their presence or absence. Let $x_i$ be the multinomial model feature vector for the $i_{th}$ document $D_i$. The $t_{th}$ element of $x_i$, written $x_{it}$, is the count of the number of times word $w_t$ occurs in document $D_i$. Thus $n_i= \sum_t x_{it}$ is the total number of words in document $D_i$.

Let $p(w_t \mid C)$ again be the probability of word $w_t$ occurring in a document of class $C$. This time estimated using the word
frequency information from the document feature vectors. We again make the naive Bayes assumption,
that the probability of each word occurring in the document is independent of the occurrences of
the other words. We can then write the document likelihood P(D_i|C) as a multinomial distribution, where the number of draws corresponds to the length of the document, and the proportion

## Example

In [94]:
path = 'data/sms.tsv'
sms = pd.read_table( path, header = None, names = ['label', 'message'] )
sms['label_num'] = sms['label'].map({ 'ham': 0, 'spam': 1 })

# extract the first 6 rows as example
X = sms.loc[ :6, 'message' ]
y = sms.loc[ :6, 'label_num' ]
sms.head(6)

  label                                            message  label_num
0   ham  Go until jurong point, crazy.. Available only ...          0
1   ham                      Ok lar... Joking wif u oni...          0
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...          1
3   ham  U dun say so early hor... U c already then say...          0
4   ham  Nah I don't think he goes to usf, he lives aro...          0
5  spam  FreeMsg Hey there darling it's been 3 week's n...          1


In [99]:
# retain only words that appear that least in two documents
vect = CountVectorizer( min_df = 2 )
X_dtm = vect.fit_transform(X).toarray()
print( vect.get_feature_names() )
X_dtm

['in', 'like', 'ok', 'std', 'there', 'to']


array([[1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 3],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 1, 1, 1, 1, 2],
       [0, 2, 0, 0, 0, 1]], dtype=int64)

## Reference

- [Notes on Text Classification using Naive Bayes](http://www.inf.ed.ac.uk/teaching/courses/inf2b/learnnotes/inf2b-learn-note07-2up.pdf)
- [Youtube Video on Introduction to the Multinomial Distribution](https://www.youtube.com/watch?v=syVW7DgvUaY)

http://cpmarkchang.logdown.com/posts/195584-natural-language-processing-pointwise-mutual-information




[Paper: Improved Naive Bayes on Sentiment Analysis](https://github.com/vivekn/sentiment)


[Sebastian Raschka's blog on Naive Bayes and Text Classification](http://sebastianraschka.com/Articles/2014_naive_bayes_1.html)

https://en.wikipedia.org/wiki/Naive_Bayes_classifier

https://github.com/wepe/MachineLearning/tree/master/NaiveBayes